# Task 5: Model Interpretability

This notebook handles:
- SHAP explanations
- Difficult case analysis
- Model transparency reports

In [1]:
import sys
sys.path.append('..')

import pandas as pd
from pathlib import Path
from transformers import pipeline
from src.interpretability.model_explainer import NERExplainer
from src.labeling.conll_labeler import CoNLLLabeler

In [2]:
# Load best model (from evaluation results)
results_df = pd.read_csv("../models/comparison_results.csv")
best_model_name = results_df.iloc[0]['model']
model_path = f"../models/checkpoints/{best_model_name.replace('/', '_')}"

print(f"Using best model: {best_model_name}")
print(f"Model path: {model_path}")

Using best model: xlm-roberta-base
Model path: ../models/checkpoints/xlm-roberta-base


In [3]:
# Initialize model pipeline and explainer
if Path(model_path).exists():
    model_pipeline = pipeline('ner', model=model_path, aggregation_strategy='simple')
    explainer = NERExplainer(model_pipeline)
    print("Model and explainer initialized")
else:
    print(f"Model not found at {model_path}")

Device set to use cpu


Model and explainer initialized


## Test Model Predictions

In [4]:
# Test predictions on sample texts
sample_texts = [
    "ሻንጣ ዋጋ 500 ብር አዲስ አበባ ቦሌ",
    "ስልክ በ 2000 ብር መርካቶ ላይ",
    "ጫማ እና ልብስ ፒያሳ ውስጥ"
]

if 'model_pipeline' in locals():
    for text in sample_texts:
        predictions = model_pipeline(text)
        print(f"\nText: {text}")
        print("Predictions:")
        for pred in predictions:
            print(f"  {pred['word']} -> {pred['entity_group']} ({pred['score']:.3f})")
else:
    print("Model pipeline not initialized. Run previous cell first.")


Text: ሻንጣ ዋጋ 500 ብር አዲስ አበባ ቦሌ
Predictions:

Text: ስልክ በ 2000 ብር መርካቶ ላይ
Predictions:

Text: ጫማ እና ልብስ ፒያሳ ውስጥ
Predictions:


## SHAP Explanations

In [5]:
# Generate SHAP explanations
if 'explainer' in locals():
    for text in sample_texts[:2]:
        print(f"\nExplaining: {text}")
        
        explanation = explainer.explain_with_shap(text)
        
        if explanation and 'shap_values' in explanation:
            print("SHAP values:")
            shap_vals = explanation['shap_values']
            labels = ['O', 'Product', 'Location', 'Price']
            for i, val in enumerate(shap_vals):
                print(f"  {labels[i]}: {val:.3f}")
        else:
            print("  No explanation generated")
else:
    print("Explainer not initialized. Run previous cell first.")


Explaining: ሻንጣ ዋጋ 500 ብር አዲስ አበባ ቦሌ
SHAP values:
  O: 0.250
  Product: 0.250
  Location: 0.250
  Price: 0.250

Explaining: ስልክ በ 2000 ብር መርካቶ ላይ
SHAP values:
  O: 0.250
  Product: 0.250
  Location: 0.250
  Price: 0.250


## Difficult Cases Analysis

In [6]:
# Load test data for difficult case analysis
labeler = CoNLLLabeler()
with open("../data/labeled/train_data.conll", 'r', encoding='utf-8') as f:
    conll_data = f.read()

test_data = labeler.parse_conll_data(conll_data)
print(f"Analyzing {len(test_data)} examples for difficult cases")

Analyzing 50 examples for difficult cases


In [7]:
# Analyze difficult cases
difficult_cases = explainer.analyze_difficult_cases(test_data[:20])

print(f"Found {len(difficult_cases)} difficult cases")

for i, case in enumerate(difficult_cases[:3]):
    print(f"\nDifficult Case {i+1}:")
    print(f"Text: {case['text'][:100]}...")
    print(f"Issues: {case['issues']}")
    print(f"Predictions: {len(case['predictions'])} entities")

Found 8 difficult cases

Difficult Case 1:
Text: 💥💥...................................💥💥 ❓ በረፍት ቀንዎ ሱቅ ላይ መስተናገድ ለምትፈልጉ ውድ ደንበኞቻችን 🌐ነገ ከጠዋቱ 4:30 _ ቀኑ...
Issues: ['Model over-predicting entities', "Entity type mismatch: true=set(), pred={'PRICE'}"]
Predictions: 6 entities

Difficult Case 2:
Text: 📌 ለ1 ሳምንት የሚቆይ 🎯 Nova®👟👟👚👔 አነስተኛ የልብስ እና የጫማ ማጠቢያ ማሽን 👟👔🩳👟 🔰 ለቲሸርት፡ ለጂንስ ፡ ለሹራብ፡ለአንሶላ ፣ጫማ 🔰በጠቅላላው እስ...
Issues: ["Entity type mismatch: true={'PRICE', 'Product'}, pred={'PRICE'}"]
Predictions: 2 entities

Difficult Case 3:
Text: **ORGANIC EXTRA OLIVE OIL 2L Price 6500 birr Telegram ፡-******** Msg👉 Lobelia pharmacy and cosmetics...
Issues: ['Model over-predicting entities', "Entity type mismatch: true={'PRICE'}, pred={'LOC', 'PRICE'}"]
Predictions: 9 entities


## Generate Interpretability Report

In [8]:
# Generate comprehensive report
explainer.generate_interpretability_report(test_data[:20], "../models/interpretability_report.json")

print("Interpretability report saved to models/interpretability_report.json")

Interpretability report saved to models/interpretability_report.json


## View Report Summary

In [9]:
# Load and display report summary
import json

with open("../models/interpretability_report.json", 'r', encoding='utf-8') as f:
    report = json.load(f)

print("Interpretability Report Summary:")
print(f"Cases analyzed: {report['summary']['total_cases_analyzed']}")
print(f"Difficult cases: {report['summary']['difficult_cases_found']}")

print("\nCommon Issues:")
for issue, count in report['summary']['common_issues'].items():
    print(f"  {issue}: {count}")

print("\nRecommendations:")
for rec in report['recommendations']:
    print(f"  - {rec}")

print("\nModel interpretability analysis completed!")

Interpretability Report Summary:
Cases analyzed: 20
Difficult cases: 8

Common Issues:
  Model over-predicting entities: 4
  Entity type mismatch: true={'PRICE'}, pred={'LOC', 'PRICE'}: 2
  Model failed to detect any entities: 2
  Entity type mismatch: true={'Product'}, pred=set(): 2
  Entity type mismatch: true={'LOC', 'PRICE'}, pred={'PRICE'}: 2
  Entity type mismatch: true=set(), pred={'PRICE'}: 1
  Entity type mismatch: true={'PRICE', 'Product'}, pred={'PRICE'}: 1

Recommendations:
  - Consider increasing model sensitivity or adding more training data with similar patterns.
  - Consider adjusting confidence thresholds or adding negative examples to training data.
  - Review entity type definitions and add more diverse examples for each entity type.
  - Consider data augmentation techniques for underrepresented entity patterns.

Model interpretability analysis completed!
